## Importing libraries

In [10]:
import torch
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [3]:
pandas2ri.activate()

fname_graph = "test_save_df.rds"
fname_param = "trees-dataset/ntrees-10000-ntaxa-100-1000-lambda-0.1-1-epsilon-0-0.9-q-0.01-0.1-sscheck-TRUE-param.rds"

readRDS = robjects.r['readRDS']
df_graph = readRDS(fname_graph)
df_graph = pandas2ri.rpy2py(df_graph)
df_param = readRDS(fname_param)
df_param = pandas2ri.rpy2py(df_param)

by .GlobalEnv when processing object ‘test_dl’
by .GlobalEnv when processing object ‘test_dl’
by .GlobalEnv when processing object ‘dnn.fit’
by .GlobalEnv when processing object ‘dnn.fit’


In [4]:
n_param = len(df_param)
n_trees = len(df_graph)

In [65]:
def convert_df_to_tensor(df_node, df_edge, params):

    n_node, n_edge = df_node.shape[0], df_edge.shape[0]

    l1, l2 = [], []
    
    for i in range(n_edge):
        edge = df_edge.iloc[i]
        u, v = edge[0]-1, edge[1]-1
        l1 = l1 + [u,v]
        l2 = l2 + [v,u]

    edge_index = torch.tensor([l1,l2], dtype=torch.long)

    x = []

    for i in range(n_node):
        node_attr = list(df_node.iloc[i])
        x.append(node_attr)

    x = torch.tensor(x, dtype = torch.float)

    y = torch.tensor(params, dtype = torch.float)

    data = Data(x = x, edge_index = edge_index, y = y)

    return(data)

In [66]:
data_list = []

for n in range(n_trees):
    df_node, df_edge = df_graph[n][0], df_graph[n][1]
    params = [df_param[i][n] for i in range(n_param)]
    data = convert_df_to_tensor(df_node, df_edge, params)
    data_list.append(data)

dataloader = DataLoader(data_list, batch_size=1)

In [34]:
x = torch.tensor([[-10,1],[-6,1],[0,1]], dtype=torch.float)
edge_index = torch.tensor([[0,1,1,2],
                           [1,0,2,1]], dtype=torch.long)
#print(edge_index.size())
#edge_attr = torch.tensor([[4],[4],[6],[6]])
y = torch.tensor([1,2], dtype=torch.float)

data = Data(x=x, edge_index=edge_index, y=y)
#device = "cuda:0"
#data.to(device=device)

In [70]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 16)
        self.conv2 = GCNConv(16, 6)
        #self.lin   = torch.nn.Linear(8,6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        #flatten = torch.nn.Flatten(start_dim=-2)
        #x = flatten(x)
        #x = self.lin(x)

        return x

In [19]:
data.num_nodes

1851

In [72]:
device = "cuda:2"
model = GCN()#.to(device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(2):
    for data in dataloader:
        data#.to(device=device)
        optimizer.zero_grad()
        out = model(data)#.to(device = device)
        loss = F.mse_loss(out, data.y)
        loss.backward()
        optimizer.step()
    print(loss)

/tmp/ipykernel_28749/428982251.py:11: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([1851, 6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(out, data.y)
/tmp/ipykernel_28749/428982251.py:11: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([1339, 6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(out, data.y)
/tmp/ipykernel_28749/428982251.py:11: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([1341, 6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(out, data.y)
/tmp/ipykernel_28749/428982251.py:11: UserWarning: Using a target size (torch.Size([6])) that is different to the input size (torch.Size([1103

tensor(5.6296, grad_fn=<MseLossBackward0>)
tensor(0.8253, grad_fn=<MseLossBackward0>)


In [68]:
model = GCN()
model(data)

torch.Size([1443, 4]) torch.Size([2, 2884])


tensor([[ 69.9019, -14.4969,  29.7104,   0.3676, -17.1890, -37.8881],
        [ 58.0165, -32.0818,  35.2376, -52.1253,  48.8288, -99.8259],
        [ 34.2155, -13.0467,   8.7604, -25.1159,  19.2250, -40.2748],
        ...,
        [ 20.1410, -15.2125,  27.5863,  -6.3295,  -1.3155, -46.5349],
        [ 19.6350, -11.4280,  23.9995,  -7.3664,   1.2568, -52.1220],
        [ 19.8630, -14.3157,  24.5136,  -6.0863,  -1.6879, -49.7893]],
       grad_fn=<AddBackward0>)

In [54]:
x = torch.rand((18,4), dtype=torch.float)
l = [i for i in range(3700)]
edge_index = torch.tensor([l,l], dtype=torch.long)
y = torch.tensor([1,2,4,5,6,7], dtype=torch.float)

data = Data(x = x, edge_index = edge_index, y = y)
print(data)

model = GCN()
model(data)

Data(x=[18, 4], edge_index=[2, 3700], y=[6])
torch.Size([18, 4]) torch.Size([2, 3700])


IndexError: index 1850 is out of bounds for dimension 0 with size 18

In [119]:
flatten = torch.nn.Flatten(start_dim=-2)
flatten(out)

tensor([ 0.8202, -1.3130,  0.9172, -1.0637,  0.6312, -0.1265], device='cuda:0',
       grad_fn=<ReshapeAliasBackward0>)